In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys
sys.path.insert(0, './gdrive/My Drive/module')

In [0]:
import pandas as pd
import numpy as np
import functions as f1
import selfmadefunctions as f2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
path = './gdrive/My Drive/data/'
meta = 'item_metadata.csv'
train = 'train.csv'
test = 'test.csv'
many = 'many.csv'
prop = 'df_prop.csv'

In [0]:
sorted_many = pd.read_csv(path + many).drop('Unnamed: 0',axis = 1)
df_test = pd.read_csv(path + test)
df_prop = pd.read_csv(path + prop)

# time stamp
1. df_many (즉 clickout이 2번 이상인 session_id들의 df)가 산 clickout item과 cosine similarity 가 높은 순으로 impressions을 sorting한다. 
* 문제 1: df_prop에 item이 없는 경우 -> 해결: 없는 item은 일단 제외
* 문제 2: clickout을 n번 이상한 경우 한 session_id 당 (n * 25)의 cos-sim을 구해야하는데 컴퓨터 파워가 딸린다. -> 해결: clickout 횟수 가장 많은 item을 하나만 골라서 cos-sim구하자

## step 1 Clickout 2번 이상한 session_id에서 cilckout 횟수 가장 높은 clickout하나 고르기

In [0]:
def many(df):
  s = (df.loc[df.action_type == 'clickout item'].groupby('session_id').action_type.count() != 1)
  many_sess_index = s.index[s].to_list() # return session_id index
  df_many = df.loc[df.session_id.isin(many_sess_index)]
  
  return df_many

df_many = many(df_test)

In [0]:
df_notar = df_test[~df_test.index.isin(df_target.index)]

In [0]:
df_clickout = df_notar.loc[df_notar.action_type == 'clickout item']

In [0]:
df_clickout['reference_int'] = df_clickout.reference.apply(f2.only_number)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
df_interact = df_notar.loc[df_notar.action_type != 'clickout item']

In [0]:
df_interact['reference_int'] = df_interact.reference.apply(f2.only_number)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
df_interact

In [0]:
df_ref_count_interact = f2.reference_count(df_interact)

In [0]:
df_clickout_count = f2.reference_count(df_clickout)

In [0]:
# unique clickout 
df_clickout_count = df_clickout_count.groupby('session_id').first()

In [0]:
df_new = df_ref_count_interact.merge(df_clickout_count.drop('ref_count', axis = 1).rename(columns = {"reference_int" : "clickout"}), left_on = ['session_id', 'reference_int'], right_on = ['session_id', 'clickout'], how = 'left')

In [0]:
df_new

In [0]:
# interact == clickout만 있는 경우
# interact != clickout이 하나라도 있는 session을 제외한 session
df = df_new.groupby('session_id')['reference_int', 'clickout'].count()
sess_noinfo = np.array(df.loc[df.reference_int == df.clickout].index)

In [0]:
sess_noinfo

array(['0006fad60bc56', '000c98c2ace32', '000ec94dfe2fa', ...,
       'fffe54dd22bff', 'ffff0ad1c618d', 'mc56p18be6c2g'], dtype=object)

In [0]:
# interact == clickout 이고 clickout을 unique로 만든 것 중 target만 뽑아낸 것 
df_noinfo_target = f1.get_submission_target(df_test.loc[df_test.session_id.isin(sess_noinfo)])

In [0]:
df_pivot = df_new.loc[df_new.session_id.isin(sess_noinfo)].drop(['reference_int','ref_count'],axis = 1)

In [0]:
df_noinfo_target # 모든 interaction == clickout 인 session_id & unique clickout (1만4천개로 줄었다.)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
806,0MQS09UWZ4TU,d80dc1f57e556,1541616687,5,clickout item,NaN,BR,"Tibau do Sul, Brazil",mobile,NaN,513401|1700081|479176|1496061|1045504|1627077|...,50|67|41|45|50|58|40|151|42|57|51|55|160|59|11...
881,0OPI22DCY9N6,a67e7e91205e9,1541697878,39,clickout item,NaN,FR,"Calvi, France",desktop,NaN,4030032|493996|1650929|2707574|122383|3056634|...,60|70|88|68|49|56|57|56|61|59|251|264|613|129|...
1198,0XK5WT27R2MO,7eeee456d394f,1541664783,18,clickout item,NaN,HK,"Yuchi Township, Taiwan",mobile,NaN,1319134|2207104|3760984|2296040|7128054|159250...,230|120|127|279|81|129|148|60|57|77|97|105|78|...
1301,1KHD6E75J8AO,ded480bc05f23,1541618131,6,clickout item,NaN,CA,"Seven Mile Beach, Cayman Islands",mobile,NaN,1189604|713051|363706|1433968|1298756|140993|5...,100|155|637|344|223|471|812|655|258|306|344|10...
1630,28AA5X54IRCN,3ab76c99e849f,1541717205,4,clickout item,NaN,UK,"New York, USA",mobile,NaN,8764|8816|8796|8744|149587|8640|1811137|8791|8...,226|538|139|301|306|239|240|283|363|416|249|19...
1932,2T1N17V2P4DR,4a15205ca1d0a,1541614836,7,clickout item,NaN,US,"Anchorage, USA",desktop,NaN,5962952|79405|71890|1247369|9966356|68857|2393...,51|46|52|45|52|69|66|61|110|66|62|83|144|200|6...
1944,2TE5D3CL8KDN,9b2732200af43,1541555252,5,clickout item,NaN,IN,"Mysore, India",mobile,NaN,5655860|8901942|6827724|3113358|2889110|110469...,29|34|28|22|23|29|21|78|82|48|48|62|66|41|24|1...
2077,3GK9RB8M87TH,592d27abe2c44,1541589591,21,clickout item,NaN,UK,"Rome, Italy",desktop,NaN,17488|1967015|17520|150215|45375|1439271|35494...,204|182|176|182|132|116|461|231|287|230|166|15...
2265,3Z4BMMC328DA,e51956efd0243,1541616542,11,clickout item,NaN,BR,"Natal, Brazil",mobile,NaN,109326|1710829|326906|104985|477846|117545|899...,155|229|186|170|74|117|131|97|124|101|124|72|7...
2763,5AALTGQEG5DB,41662dc20a36a,1541675357,7,clickout item,NaN,JP,"Kurashiki, Japan",desktop,NaN,1532793|7566644|493201|1016659|2384134|1016621...,146|191|170|118|434|672|166


In [0]:
df = df_noinfo_target.merge(df_pivot, on = 'session_id') # 오른쪽 row개수대로 자동으로 왼족 row 개수 복제해준다! wow

In [0]:
df_expl = f1.explode(df, "impressions")

In [0]:
df_expl # 32만3천 cos-sim계산

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,prices,clickout,impressions
0,0MQS09UWZ4TU,d80dc1f57e556,1541616687,5,clickout item,NaN,BR,"Tibau do Sul, Brazil",mobile,NaN,50|67|41|45|50|58|40|151|42|57|51|55|160|59|11...,513401.0,513401
1,0MQS09UWZ4TU,d80dc1f57e556,1541616687,5,clickout item,NaN,BR,"Tibau do Sul, Brazil",mobile,NaN,50|67|41|45|50|58|40|151|42|57|51|55|160|59|11...,513401.0,1700081
2,0MQS09UWZ4TU,d80dc1f57e556,1541616687,5,clickout item,NaN,BR,"Tibau do Sul, Brazil",mobile,NaN,50|67|41|45|50|58|40|151|42|57|51|55|160|59|11...,513401.0,479176
3,0MQS09UWZ4TU,d80dc1f57e556,1541616687,5,clickout item,NaN,BR,"Tibau do Sul, Brazil",mobile,NaN,50|67|41|45|50|58|40|151|42|57|51|55|160|59|11...,513401.0,1496061
4,0MQS09UWZ4TU,d80dc1f57e556,1541616687,5,clickout item,NaN,BR,"Tibau do Sul, Brazil",mobile,NaN,50|67|41|45|50|58|40|151|42|57|51|55|160|59|11...,513401.0,1045504
5,0MQS09UWZ4TU,d80dc1f57e556,1541616687,5,clickout item,NaN,BR,"Tibau do Sul, Brazil",mobile,NaN,50|67|41|45|50|58|40|151|42|57|51|55|160|59|11...,513401.0,1627077
6,0MQS09UWZ4TU,d80dc1f57e556,1541616687,5,clickout item,NaN,BR,"Tibau do Sul, Brazil",mobile,NaN,50|67|41|45|50|58|40|151|42|57|51|55|160|59|11...,513401.0,477916
7,0MQS09UWZ4TU,d80dc1f57e556,1541616687,5,clickout item,NaN,BR,"Tibau do Sul, Brazil",mobile,NaN,50|67|41|45|50|58|40|151|42|57|51|55|160|59|11...,513401.0,513656
8,0MQS09UWZ4TU,d80dc1f57e556,1541616687,5,clickout item,NaN,BR,"Tibau do Sul, Brazil",mobile,NaN,50|67|41|45|50|58|40|151|42|57|51|55|160|59|11...,513401.0,5115914
9,0MQS09UWZ4TU,d80dc1f57e556,1541616687,5,clickout item,NaN,BR,"Tibau do Sul, Brazil",mobile,NaN,50|67|41|45|50|58|40|151|42|57|51|55|160|59|11...,513401.0,6032676


In [0]:
# 문제 : df_prop에 없는 item_id가 있다. 이 경우 cossim 안구해짐. 이 item id를 df_expl에서 제거해야한다. 

# 8개 제외
in_clickout = set(df_expl.clickout) - (set(df_expl.clickout) - (set(df_prop.item_id) & set(df_expl.clickout)))

# 120개 제외
in_ref = set(df_expl.impressions) - (set(df_expl.impressions) - (set(df_prop.item_id) & set(df_expl.impressions)))

In [0]:
df_expl = df_expl.loc[(df_expl.clickout.isin(in_clickout)) & (df_expl.impressions.isin(in_ref))]

## Step 2  cos-sim 구하기

In [0]:
# 1 using APPLY (*RECOMMENDED!)
def cos(pivot, comp):
  pivot = np.delete(np.array(df_prop.loc[df_prop.item_id == pivot]), [0]).reshape(1,-1)
  comp = np.delete(np.array(df_prop.loc[df_prop.item_id == comp]), [0]).reshape(1,-1)
  cosine_similarity(pivot,comp).flatten()
  
df_expl['similarity'] = df_expl.apply(lambda row : cos(df_expl['clickout'], df_expl['impressions']), axis = 1)

ValueError: ignored

In [0]:
# 2. using FOR (오래걸림매우매우 *지양)
from tqdm import tqdm_notebook

In [0]:
tempt = zip(df_expl.clickout,df_expl.impressions)

In [0]:
start = np.array([])
for x,y in tqdm_notebook(tempt):
  pivot = np.delete(np.array(df_prop.loc[df_prop.item_id == x]), [0]).reshape(1,-1)
  comp = np.delete(np.array(df_prop.loc[df_prop.item_id == y]), [0]).reshape(1,-1)
  start = np.hstack([start, cosine_similarity(pivot,comp).flatten()])
  

In [0]:
start 

array([1.        , 0.46659064, 0.51499152, ..., 0.69566559, 0.86755369,
       0.78693588])

In [0]:
df_expl['similarity'] = start

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
df_sub = df_expl[['session_id','impressions', 'similarity']]

In [0]:
df_sub

,session_id,impressions,similarity
0,d80dc1f57e556,513401,1.000000
1,d80dc1f57e556,1700081,0.466591
2,d80dc1f57e556,479176,0.514992
3,d80dc1f57e556,1496061,0.624500
4,d80dc1f57e556,1045504,0.526135
5,d80dc1f57e556,1627077,0.546231
6,d80dc1f57e556,477916,0.421057
7,d80dc1f57e556,513656,0.695781
8,d80dc1f57e556,5115914,0.680545
9,d80dc1f57e556,6032676,0.657494


In [0]:
GR_COLS = ["user_id", "session_id", "timestamp","step"]
df_out = df_expl.assign(impressions = lambda x: x["impressions"].apply(str)).sort_values(GR_COLS + ["similarity"], ascending = [True, True,True, True, False])

In [0]:
df_out = f1.group_concat(df_out, GR_COLS, "impressions")
df_out.rename(columns = {"impressions" : "item_recommendations"}, inplace = True)
df_out

,user_id,session_id,timestamp,step,item_recommendations
0,002CUWLQDUV7,df9bd7cb3dbac,1541620323,4,16667 16790 52692 16721 51800 6264312 4936930 ...
1,003XRCCDD37S,da615a2710903,1541608168,3,1295070 2898744 2606906 2644004 4962844 277455...
2,008WVHQ5AMHZ,9949978200674,1541673098,3,52220 52980 53677 51288 2133708 1384302 162227...
3,00AT1J5PQY60,805c3d370312e,1541578458,5,5164008 2714522 6622368 80218 37520 98942 2417...
4,00CBN9FD9NSM,0a90d6ed14c0b,1541639087,17,3244244 2777115 3893112 3243338 2125674 404414...
5,00D9SRYXPQ96,f9e1c9d7b9d2b,1541634340,10,158343 159742 1610595 1909147 1913657 6563298 ...
6,00E0MO98FD6I,6380897ff5684,1541563256,3,509846 960131 3762536 1081912 927279 3133242 1...
7,00G1USQ1K5U3,b5ecd4e995f5a,1541705107,4,1962553 2902020 2031019 2902388 4103288 267803...
8,00GITS8Y93FY,d1669863c815a,1541678919,3,4745554 2677804 2197340 2568682 2032093 149605...
9,00HQ67Y7PBX6,b967be66f43d7,1541720129,3,8653 890283 1504633 8813 8812 8771 4593172 870...


## Step 3 이전 submission에서 해당되는 열만 값 바꾸기 

In [0]:
sub = pd.read_csv(path + '190506.csv')

In [0]:
sub.shape

(253573, 5)

In [0]:
sub_origin = sub.iloc[sub.index.difference(sub.loc[sub.session_id.isin(df_out.session_id)].index)]

In [0]:
df_sub = pd.concat([sub_origin, df_out])

In [0]:
df_sub.sort_values(["user_id"], ascending = True, inplace = True)

In [0]:
df_sub.to_csv(path + "190512.csv", index = False)